In [1]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


#### Visualise raw point cloud

In [80]:

pcd = o3d.io.read_point_cloud("./data/fragment.ply")
print(pcd)
print(np.asarray(pcd.points))
# Print min and max coordinates
print("Min:", np.min(pcd.points, axis=0))
print("Max:", np.max(pcd.points, axis=0))

# visualise raw point cloud 
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024]) 

PointCloud with 196133 points.
[[0.65234375 0.84686458 2.37890625]
 [0.65234375 0.83984375 2.38430572]
 [0.66737998 0.83984375 2.37890625]
 ...
 [2.00839925 2.39453125 1.88671875]
 [2.00390625 2.39488506 1.88671875]
 [2.00390625 2.39453125 1.88793314]]
Min: [0.55859375 0.83203125 0.56663716]
Max: [3.96608973 2.42747617 2.55859375]


### Voxel downsampling

- Lidar scans can be very large with millions of points, to improve efficiency, we reduce the overall number of points in the point cloud while retaining the overall structure and shape

In [72]:
voxel_size = 0.001
downpcd = pcd.voxel_down_sample(voxel_size)
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024]) 

### Vertex normal estimation

- `estimate_normals` function finds adjacent points and calculates the principal axis of the adjacent points using covariance analysis
    - `radius` = search radius, `max_nn` = maximum nearest neighbour
    - so radius = 0.1 (10cm search radius) and max_nn = 30 (only considers 30 neighbours)
- estimates normal vector (vector perpindicular to tangent plane) for each point in point cloud.
- sensitive to noise and variations & computationally expensive

In [73]:
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024],
                                  point_show_normal=True) 

#### Estimated vertex normal

In [74]:
downpcd.normals[0]

array([-0.38797148,  0.19045527, -0.90177876])

In [75]:
help(downpcd)

Help on PointCloud in module open3d.cpu.pybind.geometry object:

class PointCloud(Geometry3D)
 |  PointCloud class. A point cloud consists of point coordinates, and optionally point colors and point normals.
 |  
 |  Method resolution order:
 |      PointCloud
 |      Geometry3D
 |      Geometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(...)
 |      __add__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __copy__(...)
 |      __copy__(self: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __deepcopy__(...)
 |      __deepcopy__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: dict) -> open3d.cpu.pybind.geometry.PointCloud
 |  
 |  __iadd__(...)
 |      __iadd__(self: open3d.cpu.pybind.geometry.PointCloud, arg0: open3d.cpu.pybind.geometry.PointCloud) -> open3d.cpu.pybind.geometry.Point

In [76]:
print("Normal vectors of the first 10 points")
print(np.asarray(downpcd.normals)[:10, :])

Normal vectors of the first 10 points
[[-0.38797148  0.19045527 -0.90177876]
 [ 0.72307942 -0.6489038   0.23681217]
 [-0.23860851 -0.03685825 -0.97041612]
 [-0.89395443 -0.00799941 -0.44808647]
 [-0.96677645  0.09009592  0.23921961]
 [-0.12638364  0.00671902 -0.99195868]
 [-0.33108488  0.01692308 -0.94344921]
 [-0.08228643 -0.97649518  0.19921371]
 [-0.3123343  -0.02528463 -0.9496357 ]
 [ 0.29891333  0.1042068  -0.94857354]]


### Point cloud distance

- `compute_point_cloud_distance` computes the distance from source to target point cloud (only rly works with 2 sets of PCs)

In [77]:
# dists = pcd.compute_point_cloud_distance(downpcd)
# dists = np.asarray(dists)
# ind = np.where(dists > 0.01)[0]
# pcd = pcd.select_by_index(ind)
# o3d.visualization.draw_geometries([pcd],
#                                   zoom=0.3412,
#                                   front=[0.4257, -0.2125, -0.8795],
#                                   lookat=[2.6172, 2.0475, 1.532],
#                                   up=[-0.0694, -0.9768, 0.2024]) 

### bounding box

In [93]:
# aabb = downpcd.get_axis_aligned_bounding_box()
# aabb.color = (1, 0, 0)

# # Compute the oriented bounding box
# obb = downpcd.get_oriented_bounding_box()
# obb.color = (0,1,0)  # green

bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(0.5,0.6,0.5), max_bound=(3,4,2))

# Crop the point cloud
cropped_pcd = pcd.crop(bbox)

# Visualize the cropped point cloud
o3d.visualization.draw_geometries([cropped_pcd])

# Display the point cloud with its bounding boxes
# o3d.visualization.draw_geometries([downpcd, aabb, obb])

### dbscan clustering

- grouping point clouds together, density based clustering algorithm -- `eps` = distance to neighbours in cluster, `min_points` = min number of points to form cluster

In [100]:
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps = 0.02, min_points=10, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))


[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 10
point cloud has 10 clusters
